# Recherche d'Information et traitement de données massives

## Lab 9 : Apprentissage et Recherche d'information
## Learning to Rank : Approches par paires et Représentations distribuées 


L'objectif de cette séance est double. Une première partie consistera en la mise en oeuvre de l'approche d'apprentissage pour l'ordonnancement par paires de préférences et la deuxième partie portera sur l'utilisation d'approches d'apprentissage de représentations distribuées pouvant être utilisées pour faire de l'expansion de requêtes.


#### Préparation de l'environnement.

Plusieurs bibliothèques sont nécessaires pour ce TP. 
Les commandes ci-dessous vous permettent de les installer et de les importer

In [ ]:
! pip install numpy
! pip install scipy
! pip install matplotlib
! pip install scikit-learn 

In [ ]:
import itertools
import numpy as np
from scipy import stats


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn import datasets
from sklearn import svm,linear_model

### Partie 1 : Learning to rank - Approches par paires
***



##### Génération d'un jeu de données synthétique

Pour ce TP et afin de bien comprendre le principe de la transformée par paires, nous allons commencer par travailler sur un jeu de données synthétiques qu'il faudra générer.
En particulier, nous allons nous placer dans un cas où nous considérons un ensemble de 60 échantillons $X$ décrit selon 2 dimensions et dont les valeurs de pertinence sont entières, i.e. $Y = \{0, 1, 2\}$. 

Nous considérerons que ces données appartiennent à deux requêtes en divisant l'échantillon en deux parties égales telles que $X = X_1 \cup X_2$. Chaque partie est générée selon des distributions normales différentes (utilisation de la fonction [`randn`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randn.html?highlight=randn#numpy.random.randn) de numpy).

Le code ci-dessous permet de générer ce jeu de données.

In [ ]:
np.random.seed(0)
theta = np.deg2rad(60)
w = np.array([np.sin(theta), np.cos(theta)])
K = 20
X = np.random.randn(K, 2)
y = [0] * K
for i in range(1, 3):
    X = np.concatenate((X, np.random.randn(K, 2) + i * 4 * w))
    y = np.concatenate((y, [i] * K))

# Petit déplacement des éléments de la seconde partition
X[::2] -= np.array([3, 7]) 
# Création des partitions
blocks = np.array([0, 1] * (X.shape[0] / 2))

In [ ]:
# Affichage de X
X

In [ ]:
# Affichage de y 
y

Ce jeu de données va maintenant être séparé en jeu de données d'apprentissage et jeu de données de test à l'aide de scikit-learn et de son module `model_selection`.

In [ ]:
cv = StratifiedShuffleSplit(test_size=.5)
cv.get_n_splits(X, y)
for train_index, test_index in cv.split(X,y):
    X_train,X_test=X[train_index],X[test_index]
    y_train,y_test=y[train_index],y[test_index]
    b_train,b_test=blocks[train_index],blocks[test_index]

idx = (b_train == 0)

print(X_train[idx])
print(X_train[idx,0])
y_train[idx]

Nous allons maintenant afficher graphiquement le jeu de données d'apprentissage en utilisant des ronds pour les échantillons de X1 et des trianges pour les échantillons de X2.

In [ ]:
idx = (b_train == 0)
plt.scatter(X_train[idx, 0], X_train[idx, 1], c=y_train[idx], 
    marker='^', cmap=plt.cm.Reds,s=100)
plt.scatter(X_train[~idx, 0], X_train[~idx, 1], c=y_train[~idx],
    marker='o', cmap=plt.cm.Reds, s=100)
plt.show()


Faites de même pour le jeu de test.

In [ ]:
# A completer


**Que remarquez-vous ?** 


In [ ]:
# A completer


Appliquer le code ci-dessous. Cela devrait faciliter votre réponse à la question précédente.

In [ ]:
idx = (b_train == 0)
plt.scatter(X_train[idx, 0], X_train[idx, 1], c=y_train[idx], 
    marker='^', cmap=plt.cm.Reds,s=100)
plt.scatter(X_train[~idx, 0], X_train[~idx, 1], c=y_train[~idx],
    marker='o', cmap=plt.cm.Reds, s=100)
plt.arrow(0, 0, 8 * w[0], 8 * w[1], fc='gray', ec='gray', 
    head_width=0.5, head_length=0.5)
plt.text(0, 1, '$w$', fontsize=20)
plt.arrow(-3, -8, 8 * w[0], 8 * w[1], fc='gray', ec='gray', 
    head_width=0.5, head_length=0.5)
plt.text(-2.6, -7, '$w$', fontsize=20)
plt.axis('equal')
plt.show()

Faites le même travail pour le jeu de test

In [ ]:
# A completer

##### Estimation de w
Nous allons essayer d'estimer $w$ à l'aide d'un modèle linéaire. En particulier, nous vous demandons pour cela d'utiliser une régression régularisée (méthode [`linear_model.Ridge`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) de scikit_learn).

In [ ]:
# A completer


Afficher le vecteur estimé avec les données.

In [ ]:
# A completer


**Que remarquez-vous ?** Quelles sont  vos conclusions ?

In [ ]:
# A completer


##### Estimation de la qualité du modèle obtenu.


Pour évaluer la qualité de notre modèle, nous devons définir une note de classement. Comme c'est l'ordre des données qui nous intéresse, il est naturel de choisir une mesure qui compare l'ordre de notre modèle à l'ordre donné. Pour cela, nous utiliserons le coefficient de corrélation [`tau de Kendall`](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), qui est défini comme $\frac{P - Q}{P + Q}$, $P$ étant le nombre de paires concordantes et $Q$ le nombre de paires discordantes. Cette mesure est largement utilisée dans la littérature de classement (voir par exemple cet [article](http://www.cs.cornell.edu/people/tj/publications/joachims_02c.pdf)). 

Cette mesure est définie dans scipy [stats.kendalltau](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kendalltau.html)

Appliquer cette mesure pour évaluer le modèle linéaire appris sur chaque bloc séparemment et en utilisant le jeu de données de test.

In [ ]:
# A completer


**Qu'en deduisez-vous ?**

In [ ]:
# A completer


#### Transformée par paires

Comme nous l'avons vu dans le cours et comme cela a été montrée dans le [papier de Herbrich](https://www.mendeley.com/catalogue/a2709c3c-0705-3058-89db-28aeab2161f2/), si l'on considère des fonctions de classement linéaire, le problème de classement peut être transformé en un problème de classement à deux classes. Pour cela, nous formons la différence de tous les éléments comparables de telle sorte que nos données sont transformées en $(x'_k, y'_k) = (x_i - x_j, signe(y_i - y_j))$ pour toutes les paires comparables.

De cette façon, nous nous ramenons à un problème de classement à deux classes. 

Appliquer cette transformation à votre jeu de données. Nous vous recommandons pour cela d'utiliser le module [`itertools`](https://docs.python.org/3.7/library/itertools.html#itertools.combinations).

In [ ]:
# A completer



Afficher le nouveau jeu de données ainsi constitué en apprentissage et en test

In [ ]:
# A completer


**Que remarquez-vous ?**

#### Estimation du classifieur

Nous allons maintenant utiliser un SVM sur les données transformées. Nous utiliserons pour cela le module [`svm`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) de sklearn. 

Appliquer cela à votre nouveau jeu de données d'apprentissage et récupérer les valeurs du $w$ estimé.

In [ ]:
# A completer



Afficher les données d'apprentissage initiales avec le coefficient estimé $\hat{w}$ par votre modèle.Ce modèle est connu sous le nom de [RankSVM](https://en.wikipedia.org/wiki/Ranking_SVM). 

In [ ]:
# A completer


Il ne vous reste plus qu'à appliquer la mesure de Kendall à ce nouveau modèle.

In [ ]:
# A completer


**Que remarquez-vous ?**

In [ ]:
# A completer

#### Approche par paires sur le jeu de données du Lab8 (optionnel)

Il s'agit ici de mettre en place l'approche par paires sur le jeu de données du Lab8. La principale difficulté est de mettre en place la transformation dite pairwise qui permet de représenter vos données comme des paires de préférence. 

Ecrire le code permettant de transformer vos données pour pouvoir utiliser cette approche. Vous pourrez pour cela utiliser la bibliothèque scikit-learn et [itertools](https://docs.python.org/fr/3/library/itertools.html) pour créer facilement vos différentes paires.
Il suffira ensuite d'utiliser un séparateur linéaire de type SVM comme dans l'exercice précédent avec les données synthétiques.

In [ ]:
# A compléter

### Partie 2 : Représentations distribuées (source : l'excellent cs224n)
***

L'objectif de cette partie est de mettre en oeuvre les approches dites de plongement lexical et qui permettent de mieux capturer la sémantique des mots et ainsi d'améliorer la mise en correspondance entre requêtes et documents.
Pour commencer, installer les modules nécessaires et configurer votre environnement. Nous utiliserons notamment ici la bibliothèque [`gensim`](https://radimrehurek.com/gensim/)

In [ ]:
!pip3 install gensim
!pip3 install nltk

In [ ]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('reuters')



In [ ]:
import sys
assert sys.version_info[0]==3
assert sys.version_info[1] >= 5


In [ ]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
import pprint
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
import nltk
from nltk.corpus import reuters
import numpy as np
import random
import scipy as sp
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA

START_TOKEN = '<START>'
END_TOKEN = '<END>'

np.random.seed(0)
random.seed(0)
# ----------------

#### Approche 1 : A l'aide du comptage de mots


La plupart des modèles de plongement de mots partent de l'idée suivante :

*You shall know a word by the company it keeps* ([Firth, J. R. 1957:11](https://en.wikipedia.org/wiki/John_Rupert_Firth))

Ainsi la plupart des approches sont motivées par l'idée que des mots similaires, c'est-à-dire des synonymes (proches), seront utilisés dans des contextes similaires. En conséquence, des mots similaires seront souvent prononcés ou écrits avec un sous-ensemble de mots partagés, c'est-à-dire des contextes. En examinant ces contextes, nous pouvons essayer de développer des templates pour nos mots. Avec cette intuition, de nombreuses approches se basent donc sur le comptage des mots. Vous développerez ici l'une de ces stratégies, les matrices de cooccurrence (pour plus d'informations, voir [ici](https://medium.com/data-science-group-iitr/word-embedding-2d05d270b285)).


##### Co-Occurrence

Une matrice de cooccurrence compte la fréquence de cooccurrence des mots dans un environnement donné. Si un mot $w_i$ apparaît dans un document, nous considérons la *fenêtre contextuelle* qui entoure $w_i$. En supposant que la taille de notre fenêtre fixe est $n$, il s'agit alors des $n$ mots précédents et $n$ mots suivants dans ce document, c'est-à-dire les mots $w_{i-n} \dots w_{i-1}$ et $w_{i+1} \dots w_{i+n}$. Nous  pouvons ainsi construire une *matrice de cooccurrence* $M$, qui est une matrice symétrique mot par mot dans laquelle $M_{ij}$ est le nombre de fois que $w_j$ apparaît dans la fenêtre de $w_i$ parmi tous les documents.

Document 1: "all that glitters is not gold"

Document 2: "all is well that ends well"


|     *    | `<START>` | all | that | glitters | is   | not  | gold  | well | ends | `<END>` |
|----------|-------|-----|------|----------|------|------|-------|------|------|-----|
| `<START>`    | 0     | 2   | 0    | 0        | 0    | 0    | 0     | 0    | 0    | 0   |
| all      | 2     | 0   | 1    | 0        | 1    | 0    | 0     | 0    | 0    | 0   |
| that     | 0     | 1   | 0    | 1        | 0    | 0    | 0     | 1    | 1    | 0   |
| glitters | 0     | 0   | 1    | 0        | 1    | 0    | 0     | 0    | 0    | 0   |
| is       | 0     | 1   | 0    | 1        | 0    | 1    | 0     | 1    | 0    | 0   |
| not      | 0     | 0   | 0    | 0        | 1    | 0    | 1     | 0    | 0    | 0   |
| gold     | 0     | 0   | 0    | 0        | 0    | 1    | 0     | 0    | 0    | 1   |
| well     | 0     | 0   | 1    | 0        | 1    | 0    | 0     | 0    | 1    | 1   |
| ends     | 0     | 0   | 1    | 0        | 0    | 0    | 0     | 1    | 0    | 0   |
| `<END>`      | 0     | 0   | 0    | 0        | 0    | 0    | 1     | 1    | 0    | 0   |

**Note:** En NLP, les tokens `<START>` et `<END>` servent à representer le debut et la fin de phrases, paragraphes et documents.

Les lignes (ou colonnes) de cette matrice fournissent un type de vecteurs de mots (ceux basés sur la cooccurrence mot-mot), mais les vecteurs seront en général grands (linéaires dans le nombre de mots distincts dans un corpus).

Il est donc nécessaire de faire une étape de *réduction de la dimensionnalité*. En particulier, on peut ici utiliser une *SVD (Singular Value Decomposition)*, qui est une sorte de *PCA (Principal Components Analysis)* généralisée pour sélectionner les composantes principales de $k$ comme illustré ci-dessous.

![Image d'une SVD](./Figures/svd.png)

Cette représentation de co-occurrence à dimension réduite préserve les relations sémantiques entre les mots, par exemple *doctor* et *hospital* seront plus proches que *doctor* et *dog*. 

Quelques ressources concernant cette méthode :
+ [ici](https://davetang.org/file/Singular_Value_Decomposition_Tutorial.pdf). 
+ [ici](https://web.stanford.edu/class/cs168/l/l9.pdf) 

En pratique, il est difficile d'appliquer uneSVD complète à de grands corpus en raison de la mémoire nécessaire. Cependant, comme nous ne voulons que les composantes vectorielles les plus importantes pour des $k$ relativement petits, on pourra utiliser la [Truncated SVD] (https://en.wikipedia.org/wiki/Singular_value_decomposition#Truncated_SVD) - pour laquelle il existe des techniques raisonnablement efficaces pour les calculer de manière itérative.









##### Application à Reuters




Nous allons maintenant appliquer cette approche au corpus Reuters disponible dans `nltk`. Le corpus se compose de 10 788 documents d'actualité totalisant 1,3 million de mots. Ces documents couvrent 90 catégories et sont répartis en deux catégories : train et test. Plus de détails [ici](https://www.nltk.org/book/ch02.html). la fonction `read_corpus` ci-dessous permet de ne retirer que les articles de la catégorie "brut" (c'est-à-dire les articles d'actualité sur le pétrole, le gaz, etc). La fonction ajoute également des jetons `<START>` et `<END>` à chacun des documents, ainsi que des mots en minuscules. Vous n'avez **pas** à effectuer d'autres types de prétraitement.


In [ ]:
def read_corpus(category="crude"):
    """ Read files from the specified Reuter's category.
        Params:
            category (string): category name
        Return:
            list of lists, with words from each of the processed files
    """
    files = reuters.fileids(category)
    return [[START_TOKEN] + [w.lower() for w in list(reuters.words(f))] + [END_TOKEN] for f in files]

On peut ensuite l'appliquer et avoir une vue sur quelques documents.

In [ ]:
reuters_corpus = read_corpus()
pprint.pprint(reuters_corpus[:3], compact=True, width=100)

##### Question 1.1: Ecrire `distinct_words` 

Completer la fonction ci-dessous permettant de récupérer les mots uniques ou le vocabulaire du corpus. Vous pouvez pour cela utiliser des méthodes de python sur les listes comme décrit [ici](https://coderwall.com/p/rcmaea/flatten-a-list-of-lists-in-one-line-in-python) ou [ici](https://python-3-patterns-idioms-test.readthedocs.io/en/latest/Comprehensions.html).

De même les [Python sets](https://www.w3schools.com/python/python_sets.asp) peuvent être utiles pour supprimer les doublons.

In [ ]:
def distinct_words(corpus):
    """ Determine a list of distinct words for the corpus.
        Params:
            corpus (list of list of strings): corpus of documents
        Return:
            corpus_words (list of strings): list of distinct words across the corpus, sorted (using python 'sorted' function)
            num_corpus_words (integer): number of distinct words across the corpus
    """
    corpus_words = []
    num_corpus_words = -1
    
    # ------------------
    # Write your implementation here.


    # ------------------

    return corpus_words, num_corpus_words

Le code ci-dessous vous permet de tester votre fonction.

In [ ]:
# ---------------------
# Run this sanity check
# Note that this not an exhaustive check for correctness.
# ---------------------

# Define toy corpus
test_corpus = ["{} All that glitters isn't gold {}".format(START_TOKEN, END_TOKEN).split(" "), "{} All's well that ends well {}".format(START_TOKEN, END_TOKEN).split(" ")]
test_corpus_words, num_corpus_words = distinct_words(test_corpus)

# Correct answers
ans_test_corpus_words = sorted([START_TOKEN, "All", "ends", "that", "gold", "All's", "glitters", "isn't", "well", END_TOKEN])
ans_num_corpus_words = len(ans_test_corpus_words)

# Test correct number of words
assert(num_corpus_words == ans_num_corpus_words), "Incorrect number of distinct words. Correct: {}. Yours: {}".format(ans_num_corpus_words, num_corpus_words)

# Test correct words
assert (test_corpus_words == ans_test_corpus_words), "Incorrect corpus_words.\nCorrect: {}\nYours:   {}".format(str(ans_test_corpus_words), str(test_corpus_words))

# Print Success
print ("-" * 80)
print("Passed All Tests!")
print ("-" * 80)

##### Question 1.2:  Implementer `compute_co_occurrence_matrix` 

Écrire une méthode qui construit une matrice de cooccurrence pour une certaine taille de fenêtre $n$ (avec une valeur par défaut de 4), en considérant les mots $n$ avant et $n$ après le mot au centre de la fenêtre. Penser à utiliser `numpy`.


In [ ]:
def compute_co_occurrence_matrix(corpus, window_size=4):
    """ Compute co-occurrence matrix for the given corpus and window_size (default of 4).
    
        Note: Each word in a document should be at the center of a window. Words near edges will have a smaller
              number of co-occurring words.
              
              For example, if we take the document "<START> All that glitters is not gold <END>" with window size of 4,
              "All" will co-occur with "<START>", "that", "glitters", "is", and "not".
    
        Params:
            corpus (list of list of strings): corpus of documents
            window_size (int): size of context window
        Return:
            M (a symmetric numpy matrix of shape (number of unique words in the corpus , number of unique words in the corpus)): 
                Co-occurence matrix of word counts. 
                The ordering of the words in the rows/columns should be the same as the ordering of the words given by the distinct_words function.
            word2Ind (dict): dictionary that maps word to index (i.e. row/column number) for matrix M.
    """
    words, num_words = distinct_words(corpus)
    M = None
    word2Ind = {}
    
    # ------------------
    # Write your implementation here.


    # ------------------

    return M, word2Ind

Tester avec le code ci-dessous

In [ ]:
# ---------------------
# Run this sanity check
# Note that this is not an exhaustive check for correctness.
# ---------------------

# Define toy corpus and get student's co-occurrence matrix
test_corpus = ["{} All that glitters isn't gold {}".format(START_TOKEN, END_TOKEN).split(" "), "{} All's well that ends well {}".format(START_TOKEN, END_TOKEN).split(" ")]
M_test, word2Ind_test = compute_co_occurrence_matrix(test_corpus, window_size=1)

# Correct M and word2Ind
M_test_ans = np.array( 
    [[0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,],
     [0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,],
     [0., 1., 0., 0., 0., 0., 0., 0., 1., 0.,],
     [0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,],
     [0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,],
     [0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,],
     [1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,],
     [0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,],
     [0., 0., 1., 0., 1., 1., 0., 0., 0., 1.,],
     [1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,]]
)
ans_test_corpus_words = sorted([START_TOKEN, "All", "ends", "that", "gold", "All's", "glitters", "isn't", "well", END_TOKEN])
word2Ind_ans = dict(zip(ans_test_corpus_words, range(len(ans_test_corpus_words))))

# Test correct word2Ind
assert (word2Ind_ans == word2Ind_test), "Your word2Ind is incorrect:\nCorrect: {}\nYours: {}".format(word2Ind_ans, word2Ind_test)

# Test correct M shape
assert (M_test.shape == M_test_ans.shape), "M matrix has incorrect shape.\nCorrect: {}\nYours: {}".format(M_test.shape, M_test_ans.shape)

# Test correct M values
for w1 in word2Ind_ans.keys():
    idx1 = word2Ind_ans[w1]
    for w2 in word2Ind_ans.keys():
        idx2 = word2Ind_ans[w2]
        student = M_test[idx1, idx2]
        correct = M_test_ans[idx1, idx2]
        if student != correct:
            print("Correct M:")
            print(M_test_ans)
            print("Your M: ")
            print(M_test)
            raise AssertionError("Incorrect count at index ({}, {})=({}, {}) in matrix M. Yours has {} but should have {}.".format(idx1, idx2, w1, w2, student, correct))

# Print Success
print ("-" * 80)
print("Passed All Tests!")
print ("-" * 80)

##### Question 1.3 : Mettre en oeuvre le code "reduce_to_k_dim" 

Construire une méthode qui effectue une réduction de dimensionnalité sur la matrice à k dimensions. Utiliser la SVD pour prendre les k plus grandes composantes et produire une nouvelle matrice à k dimensions. 

**Note** : Tous les logiciels numpy, scipy et scikit-learn (`sklearn`) fournissent une *certaine* implémentation de la SVD, mais seuls scipy et sklearn fournissent une implémentation de la SVD tronquée, et seul sklearn fournit un algorithme randomisé efficace pour calculer la SVD tronquée à grande échelle. Veuillez donc utiliser [sklearn.decomposition.TruncatedSVD] (https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html).

In [ ]:
def reduce_to_k_dim(M, k=2):
    """ Reduce a co-occurence count matrix of dimensionality (num_corpus_words, num_corpus_words)
        to a matrix of dimensionality (num_corpus_words, k) using the following SVD function from Scikit-Learn:
            - http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html
    
        Params:
            M (numpy matrix of shape (number of unique words in the corpus , number of unique words in the corpus)): co-occurence matrix of word counts
            k (int): embedding size of each word after dimension reduction
        Return:
            M_reduced (numpy matrix of shape (number of corpus words, k)): matrix of k-dimensioal word embeddings.
                    In terms of the SVD from math class, this actually returns U * S
    """    
    n_iters = 10     # Use this parameter in your call to `TruncatedSVD`
    M_reduced = None
    print("Running Truncated SVD over %i words..." % (M.shape[0]))
    
        # ------------------
        # Write your implementation here.
    
    
        # ------------------

    print("Done.")
    return M_reduced

Tester votre code

In [ ]:
# ---------------------
# Run this sanity check
# Note that this is not an exhaustive check for correctness 
# In fact we only check that your M_reduced has the right dimensions.
# ---------------------

# Define toy corpus and run student code
test_corpus = ["{} All that glitters isn't gold {}".format(START_TOKEN, END_TOKEN).split(" "), "{} All's well that ends well {}".format(START_TOKEN, END_TOKEN).split(" ")]
M_test, word2Ind_test = compute_co_occurrence_matrix(test_corpus, window_size=1)
M_test_reduced = reduce_to_k_dim(M_test, k=2)

# Test proper dimensions
assert (M_test_reduced.shape[0] == 10), "M_reduced has {} rows; should have {}".format(M_test_reduced.shape[0], 10)
assert (M_test_reduced.shape[1] == 2), "M_reduced has {} columns; should have {}".format(M_test_reduced.shape[1], 2)

# Print Success
print ("-" * 80)
print("Passed All Tests!")
print ("-" * 80)

##### Question 1.4 : Afficher les plongement : "Plot_embeddings" 

Ici, vous allez écrire une fonction les plongements obtenus à l'aide de matplotlib.

Pour cet exemple, il peut être utile d'adapter [ce code] (https://www.pythonmembers.club/2018/05/08/matplotlib-scatter-plot-annotate-set-text-at-label-each-point/). Penser à cela à l'avenir et pour l'EI : regarder [la galerie Matplotlib] (https://matplotlib.org/gallery/index.html), de trouver un graphique qui ressemble un peu à ce que vous voulez, et adapter le code qu'ils donnent.

In [ ]:
def plot_embeddings(M_reduced, word2Ind, words):
    """ Plot in a scatterplot the embeddings of the words specified in the list "words".
        NOTE: do not plot all the words listed in M_reduced / word2Ind.
        Include a label next to each point.
        
        Params:
            M_reduced (numpy matrix of shape (number of unique words in the corpus , 2)): matrix of 2-dimensioal word embeddings
            word2Ind (dict): dictionary that maps word to indices for matrix M
            words (list of strings): words whose embeddings we want to visualize
    """

    # ------------------
    # Write your implementation here.


    # ------------------

Tester votre code

In [ ]:
# ---------------------
# Run this sanity check
# Note that this is not an exhaustive check for correctness.
# The plot produced should look like the "test solution plot" depicted below. 
# ---------------------

print ("-" * 80)
print ("Outputted Plot:")

M_reduced_plot_test = np.array([[1, 1], [-1, -1], [1, -1], [-1, 1], [0, 0]])
word2Ind_plot_test = {'test1': 0, 'test2': 1, 'test3': 2, 'test4': 3, 'test5': 4}
words = ['test1', 'test2', 'test3', 'test4', 'test5']
plot_embeddings(M_reduced_plot_test, word2Ind_plot_test, words)

print ("-" * 80)

##### Question 1.5 : Analyse des plongements obtenus

Nous allons maintenant rassembler toutes les parties précédente.
Il s'agit de calculer la matrice de cooccurrence avec une fenêtre fixe de 4 (la taille de fenêtre par défaut), sur le corpus "brut" (pétrole) de Reuters. Ensuite, nous utiliserons TruncatedSVD pour calculer les encastrements bidimensionnels de chaque mot. TruncatedSVD retourne U\*S, nous devons donc normaliser les vecteurs retournés, de sorte que tous les vecteurs apparaissent autour du cercle unitaire (donc la proximité est la proximité directionnelle). 

**Note** : La ligne de code ci-dessous qui effectue la normalisation utilise le concept NumPy de *Broadcasting*. Si vous ne connaissez pas le concept de Broadcasting, consultez
[Computation on Arrays : Broadcasting by Jake VanderPlas](https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computation-on-arrays-broadcasting.html).

Exécuter le code ci-dessous. Cela prendra probablement quelques secondes. 

In [ ]:
# -----------------------------
# Run This Cell to Produce Your Plot
# ------------------------------
reuters_corpus = read_corpus()
M_co_occurrence, word2Ind_co_occurrence = compute_co_occurrence_matrix(reuters_corpus)
M_reduced_co_occurrence = reduce_to_k_dim(M_co_occurrence, k=2)

# Rescale (normalize) the rows to make them each of unit-length
M_lengths = np.linalg.norm(M_reduced_co_occurrence, axis=1)
M_normalized = M_reduced_co_occurrence / M_lengths[:, np.newaxis] # broadcasting

words = ['barrels', 'bpd', 'ecuador', 'energy', 'industry', 'kuwait', 'oil', 'output', 'petroleum', 'venezuela']

plot_embeddings(M_normalized, word2Ind_co_occurrence, words)

**Analyser les resultats obtenus et notamment les regroupements obtenus** Comment pourriez-vous utiliser cela dans le cadre de la RI ?


#### Approche 2 : Vecteurs de mots basés sur des prédictions  (optionnel)

Comme rapidement éviqué en cours, des vecteurs de mots plus récents basés sur des prédictions ont montré de meilleures performances, tels que word2vec et GloVe. Ici, nous allons explorer les encastrements produits par GloVe. C.f. [l'article original de GloVe] (https://nlp.stanford.edu/pubs/glove.pdf).

Exécuter le code ci-dessous pour charger les vecteurs GloVe en mémoire. 

**Note** : Si c'est la première fois que vous exécutez ces cellules, c'est-à-dire que vous téléchargez le modèle, l'exécution prendra environ 15 minutes. Si vous avez déjà exécuté ces cellules auparavant, les relancer chargera le modèle sans le recharger, ce qui prendra environ 1 à 2 minutes.

In [ ]:
def load_embedding_model():
    """ Load GloVe Vectors
        Return:
            wv_from_bin: All 400000 embeddings, each lengh 200
    """
    import gensim.downloader as api
    wv_from_bin = api.load("glove-wiki-gigaword-200")
    print("Loaded vocab size %i" % len(wv_from_bin.vocab.keys()))
    return wv_from_bin

In [ ]:
# -----------------------------------
# Run Cell to Load Word Vectors
# Note: This will take several minutes
# -----------------------------------
wv_from_bin = load_embedding_model()

#### Réduire la dimensionnalité des plongements 
Comparons directement les plongements de GloVe à ceux de la matrice de co-occurrence. Afin d'éviter de manquer de mémoire, nous travaillerons plutôt avec un échantillon de 10000 vecteurs GloVe.
Exécutez les cellules suivantes pour :

1. Mettre 10000 vecteurs GloVe dans une matrice M
2. Exécutez reduce_to_k_dim (votre fonction SVD tronquée) pour réduire les vecteurs de 200 dimensions à 2 dimensions.

In [ ]:
def get_matrix_of_vectors(wv_from_bin, required_words=['barrels', 'bpd', 'ecuador', 'energy', 'industry', 'kuwait', 'oil', 'output', 'petroleum', 'venezuela']):
    """ Put the GloVe vectors into a matrix M.
        Param:
            wv_from_bin: KeyedVectors object; the 400000 GloVe vectors loaded from file
        Return:
            M: numpy matrix shape (num words, 200) containing the vectors
            word2Ind: dictionary mapping each word to its row number in M
    """
    import random
    words = list(wv_from_bin.vocab.keys())
    print("Shuffling words ...")
    random.seed(224)
    random.shuffle(words)
    words = words[:10000]
    print("Putting %i words into word2Ind and matrix M..." % len(words))
    word2Ind = {}
    M = []
    curInd = 0
    for w in words:
        try:
            M.append(wv_from_bin.word_vec(w))
            word2Ind[w] = curInd
            curInd += 1
        except KeyError:
            continue
    for w in required_words:
        if w in words:
            continue
        try:
            M.append(wv_from_bin.word_vec(w))
            word2Ind[w] = curInd
            curInd += 1
        except KeyError:
            continue
    M = np.stack(M)
    print("Done.")
    return M, word2Ind

In [ ]:
# -----------------------------------------------------------------
# Run Cell to Reduce 200-Dimensional Word Embeddings to k Dimensions
# Note: This should be quick to run
# -----------------------------------------------------------------
M, word2Ind = get_matrix_of_vectors(wv_from_bin)
M_reduced = reduce_to_k_dim(M, k=2)

# Rescale (normalize) the rows to make them each of unit-length
M_lengths = np.linalg.norm(M_reduced, axis=1)
M_reduced_normalized = M_reduced / M_lengths[:, np.newaxis] # broadcasting

##### Question 2.1 : Analyse des plongements GloVe

Lancez la cellule ci-dessous pour tracer les plongements GloVe en 2D pour "["barils", "bpd", "ecuador", "energy", "industry", "kuwait", "oil", "output", "petroleum", "venezuela"]`.



In [ ]:
words = ['barrels', 'bpd', 'ecuador', 'energy', 'industry', 'kuwait', 'oil', 'output', 'petroleum', 'venezuela']
plot_embeddings(M_reduced_normalized, word2Ind, words)

Qu'est-ce qui se regroupe dans l'espace bidimensionnel ? Qu'est-ce qui ne se regroupe pas et qui, selon vous, devrait l'être ? En quoi le tracé est-il différent de celui généré précédemment à partir de la matrice de cooccurrence ? Quelle est la raison possible de cette différence ?

##### Question 2.2 : Mots à sens multiples 

Les polysèmes et les homonymes sont des mots qui ont plus d'une signification (voir cette [page wiki] (https://en.wikipedia.org/wiki/Polysemy) pour en savoir plus sur la différence entre les polysèmes et les homonymes). Trouvez un mot ayant au moins deux significations différentes de sorte que les dix mots les plus similaires (selon la similarité du cosinus) contiennent des mots apparentés ayant les deux significations. For example, "leaves" a "vanishes" et "stalks" in the top 10, et "scoop" a "handed_waffle_cone" et "lowdown". Vous devrez probablement essayer plusieurs mots polysémiques ou homonymes avant d'en trouver un. Veuillez indiquer le mot que vous découvrez et les multiples significations qui figurent dans le top 10. Pourquoi pensez-vous que plusieurs des mots polysémiques ou homonymiques que vous avez essayés n'ont pas fonctionné (c'est-à-dire que les 10 mots les plus similaires du top ne contiennent qu' **une** des significations des mots) ?

**Note** : Utiliser la fonction `wv_from_bin.most_similar(word)` pour obtenir le top 10 des mots les plus similaires. Cette fonction classe tous les autres mots du vocabulaire en fonction de leur similarité cosinus avec le mot donné. Pour plus d'informations, [documentation GenSim](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.FastTextKeyedVectors.most_similar).

In [ ]:
# ------------------
    # Write your implementation here.
    
    
# ------------------

#####  Résoudre les analogies avec les vecteurs de mots

Il a été démontré que les vecteurs de mots *sont parfois* capables de résoudre des analogies. 

À titre d'exemple, pour l'analogie "homme : roi : : femme : x" (lire : l'homme est au roi comme la femme est à x), qu'est-ce que x ?

Dans la cellule ci-dessous, nous vous montrons comment utiliser les vecteurs de mots pour trouver x. La fonction "le plus similaire" trouve les mots qui sont les plus similaires aux mots de la liste "positive" et les plus différents des mots de la liste "négative". La réponse à l'analogie sera le mot classé le plus similaire (la plus grande valeur numérique).

**Note:** Une documentation supplémentaire sur la fonction `most_similar` peut être trouvée 
dans la [documentation GenSim](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.FastTextKeyedVectors.most_similar).

In [ ]:
# Run this cell to answer the analogy -- man : king :: woman : x
pprint.pprint(wv_from_bin.most_similar(positive=['woman', 'king'], negative=['man']))

##### Question 2.4 : Trouver des analogies 

Trouvez un exemple d'analogie. Dans votre solution, veuillez indiquer l'analogie complète sous la forme x:y : : a:b. 

**Note** : Vous devrez peut-être essayer de nombreuses analogies pour en trouver une qui fonctionne !

In [ ]:
    # ------------------
    # Write your implementation here.


    # ------------------